In [13]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [15]:
# Task 1 - Data Loading and Preprocessing
print("=== Task 1: Data Loading and Preprocessing ===")

# Load the Iris dataset
iris = load_iris()
X = iris.data  # Features: sepal length, sepal width, petal length, petal width
y = iris.target  # Target: species (0: setosa, 1: versicolor, 2: virginica)

# Split the dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")

# Perform feature scaling (standardization) to normalize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Feature scaling completed using StandardScaler")

# One-hot encoding equivalent using PyTorch (convert to tensors)
X_train_tensor = torch.FloatTensor(X_train_scaled)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_train_tensor = torch.LongTensor(y_train)  
y_test_tensor = torch.LongTensor(y_test)

# Create one-hot encoded versions for display (equivalent to to_categorical)
y_train_onehot = F.one_hot(y_train_tensor, num_classes=3).float()
y_test_onehot = F.one_hot(y_test_tensor, num_classes=3).float()

print(f"One-hot encoded labels shape: {y_train_onehot.shape}")
print("Data preprocessing completed successfully!\n")


=== Task 1: Data Loading and Preprocessing ===
Training set size: 120 samples
Testing set size: 30 samples
Feature scaling completed using StandardScaler
One-hot encoded labels shape: torch.Size([120, 3])
Data preprocessing completed successfully!



In [16]:
# Task 2 - Neural Network Construction
print("=== Task 2: Neural Network Construction ===")

# Define the Neural Network class
iris_net=nn.Sequential(
    nn.Linear(4,8),
    nn.ReLU(),
    nn.Linear(8,3)
)

# Create the model instance
model = iris_net

print("Neural Network Architecture:")
print(model)
print("Neural network construction completed!\n")

=== Task 2: Neural Network Construction ===
Neural Network Architecture:
Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=3, bias=True)
)
Neural network construction completed!



In [17]:
# Task 3 - Model Compilation and Training
print("=== Task 3: Model Compilation and Training ===")

# Define loss function and optimizer (equivalent to model.compile in Keras)
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters()) 

print("Model compiled with:")
print("- Optimizer: Adam")
print("- Loss function: CrossEntropyLoss (Categorical Cross-entropy)")

# Create data loaders for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False)

# Training loop
print("\nStarting model training...")
num_epochs = 100
train_losses = []
train_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_X, batch_y in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)
    
    # Print progress every 20 epochs
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

print("Model training completed!\n")

=== Task 3: Model Compilation and Training ===
Model compiled with:
- Optimizer: Adam
- Loss function: CrossEntropyLoss (Categorical Cross-entropy)

Starting model training...
Epoch [20/100], Loss: 0.3770, Accuracy: 89.17%
Epoch [40/100], Loss: 0.2292, Accuracy: 94.17%
Epoch [60/100], Loss: 0.1487, Accuracy: 95.83%
Epoch [80/100], Loss: 0.1069, Accuracy: 96.67%
Epoch [100/100], Loss: 0.0847, Accuracy: 97.50%
Model training completed!



In [18]:
print("=== Task 4: Model Evaluation ===")

# Evaluate the model on the test set
model.eval()
test_loss = 0.0
correct = 0
total = 0
all_predictions = []
all_targets = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        test_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
        
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(batch_y.cpu().numpy())

test_loss /= len(test_loader)
test_accuracy = correct / total

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# Display some predictions vs actual
print("\nSample predictions vs actual:")
print("Predicted | Actual | Species")
print("-" * 30)
for i in range(min(10, len(all_targets))):
    pred_species = iris.target_names[all_predictions[i]]
    true_species = iris.target_names[all_targets[i]]
    print(f"{pred_species:9} | {true_species:6} | {true_species}")

print(f"\nFinal Model Accuracy on Test Set: {test_accuracy*100:.2f}%")



=== Task 4: Model Evaluation ===
Test Loss: 0.1257
Test Accuracy: 0.9333 (93.33%)

Sample predictions vs actual:
Predicted | Actual | Species
------------------------------
setosa    | setosa | setosa
virginica | virginica | virginica
versicolor | versicolor | versicolor
versicolor | versicolor | versicolor
setosa    | setosa | setosa
versicolor | versicolor | versicolor
setosa    | setosa | setosa
setosa    | setosa | setosa
virginica | virginica | virginica
versicolor | versicolor | versicolor

Final Model Accuracy on Test Set: 93.33%
